## 라이브러리 선언

In [1]:
# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

In [2]:
# 머신러닝 라이브러리
from sklearn.svm import SVC # support vector classifier
from sklearn.neighbors import KNeighborsClassifier # k neighbors

In [3]:
# 데이터 전처리 라이브러리
from sklearn.model_selection import train_test_split

In [4]:
# Classification Metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
# 이 4개가 정확도 지표이다! (전의 모델이랑 달라진 점)

## 데이터 불러오기

In [5]:
csData = pd.read_csv("../dataset/customer.csv")
csData.head(2)

,balance,stock,label
0,30000000,22500000,normal
1,280000000,48000000,diamond


In [6]:
csData.label.drop_duplicates()

0     normal
1    diamond
4        vip
Name: label, dtype: object

### 1. 타입 통합 / 숫자형 컬럼 추가

In [7]:
# skip
csData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   balance  20000 non-null  int64 
 1   stock    20000 non-null  int64 
 2   label    20000 non-null  object
dtypes: int64(2), object(1)
memory usage: 468.9+ KB


In [8]:
csData.columns

Index(['balance', 'stock', 'label'], dtype='object')

In [9]:
csData = csData.astype({"balance":"int",
                        "stock":"int",
                        "label":"object"})

In [10]:
csData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   balance  20000 non-null  int32 
 1   stock    20000 non-null  int32 
 2   label    20000 non-null  object
dtypes: int32(2), object(1)
memory usage: 312.6+ KB


### 2. 특성 선정 / 데이터 분리

In [11]:
labelColumn = "label_new"

In [12]:
# 상관계수 생성 함수는 숫자형 컬럼에 대해서만 상관계수를 계산한다!
# 구하려는 label이 없다는 문제가 있다!
csData.corr()

,balance,stock
balance,1.000000,0.565942
stock,0.565942,1.000000


In [13]:
csData.label

0         normal
1        diamond
2        diamond
3         normal
4            vip
          ...   
19995    diamond
19996     normal
19997    diamond
19998    diamond
19999    diamond
Name: label, Length: 20000, dtype: object

In [14]:
# 중요한 순서대로 함.
# 좀 많다 하면 sort를 한 후 맵핑하면 된다!
# 머신러닝의 70%는 데이터전처리 기술!
labelMap = {"normal":0,
            "diamond":1,
            "vip":2}

In [15]:
# LABEL_ENCODER : 내가 원하는 값으로 맵핑하기가 힘들다
# 딕셔너리 : 내가 원하는 값으로 맵핑이 가능하다! (지금처럼)
csData["label_new"] = csData.label.map(labelMap)

In [16]:
csData.corr()

,balance,stock,label_new
balance,1.000000,0.565942,0.883144
stock,0.565942,1.000000,0.824174
label_new,0.883144,0.824174,1.000000


In [17]:
corrDf = csData.corr()

In [18]:
csData[labelColumn] = csData.label.map(labelMap)

In [19]:
corrStd = 0.5

In [20]:
features = list( corrDf.loc[ (abs(corrDf[labelColumn]) > corrStd) & 
                      (corrDf[labelColumn] != 1) ].index )

In [21]:
set(features)

{'balance', 'stock'}

In [22]:
list(labelColumn)

['l', 'a', 'b', 'e', 'l', '_', 'n', 'e', 'w']

## 2-2 데이터 분리

In [23]:
# 위에 corr()을 이용하여 관련된 feature값을 찾기위해 label_new를 사용
# 이젠 원래대로 해줘야!
labelColumn = "label"

In [26]:
trainingDataFeatures, \
testDataFeatures, \
trainingDataLabel, \
testDataLabel = \
train_test_split( csData.loc[:, features],
                  csData.loc[:, labelColumn],
                  random_state=1,
                  test_size = 0.2)

In [27]:
trainingDataLabel

7013      normal
7766      normal
5910     diamond
7020     diamond
16952    diamond
          ...   
10955    diamond
17289    diamond
5192      normal
12172        vip
235      diamond
Name: label, Length: 16000, dtype: object

In [28]:
print(trainingDataFeatures.shape)
print(testDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataLabel.shape)

(16000, 2)
(4000, 2)
(16000,)
(4000,)


### 3. 모델 선언 / 학습

In [29]:
# shift + tab : 주로 C값을 하이퍼파라미터 튜닝으로 많이 씀
model_svm = SVC(random_state=5)

In [30]:
model_svm.fit(trainingDataFeatures, trainingDataLabel)

SVC(random_state=5)

### 4. 에측

In [31]:
# 값이 1, 1, 1 이렇게 나오면 안됨. regression으로 푼 것.
model_svm.predict(testDataFeatures)

array(['diamond', 'diamond', 'diamond', ..., 'diamond', 'diamond',
       'diamond'], dtype=object)

In [32]:
predictSvm = model_svm.predict(testDataFeatures)

### 5. 데이터 정리

In [33]:
testDataAll = csData.loc[ testDataLabel.index ]

In [34]:
testDataAll["predict_svm"] = predictSvm

In [35]:
testDataAll

,balance,stock,label,label_new,predict_svm
11456,744000000,38000000,diamond,1,diamond
16528,724000000,32000000,diamond,1,diamond
3253,704000000,27333333,diamond,1,diamond
18614,240000000,30500000,normal,0,normal
1544,258000000,28500000,normal,0,normal
...,...,...,...,...,...
6375,640000000,48666666,diamond,1,diamond
14837,652000000,26000000,diamond,1,diamond
3931,564000000,45333333,diamond,1,diamond
18266,528000000,46000000,diamond,1,diamond


In [38]:
# testDataAll.drop(columns=["PREDICT_SVM"], inplace=True)

## 6. 정확도 검증

In [39]:
accuracy = accuracy_score(y_true=testDataAll.label,
                          y_pred=testDataAll.predict_svm)

In [40]:
# regression 보다 정확성이 높다. 문제가 몇 개 없어서
accuracy

0.995

In [44]:
from sklearn.metrics import classification_report

In [46]:
# 엑셀로 정리한 거랑 같은 값이 나옴!
print(classification_report(y_true=testDataAll.label,
                           y_pred=testDataAll.predict_svm))

              precision    recall  f1-score   support

     diamond       1.00      1.00      1.00      2346
      normal       0.99      0.99      0.99      1188
         vip       1.00      0.99      0.99       466

    accuracy                           0.99      4000
   macro avg       1.00      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000



In [41]:
# 구글 스프레드 시트 열어서 붙여넣기 하면 나옴 
testDataAll.to_clipboard()

In [43]:
# 저장시 index=False 항상 해주자!
# label은 정답지, predict_svm은 예측값
testDataAll.to_csv("../Session07 - 유형별로 실습하는 머신러닝/svc_result.csv", index=False) # (작업한 것 엑셀로 보기)